In [1]:
!pip install ase

     |████████████████████████████████| 2.9 MB 8.7 MB/s eta 0:00:01
     |████████████████████████████████| 9.2 MB 13.4 MB/s eta 0:00:01
     |████████████████████████████████| 34.5 MB 10.4 MB/s eta 0:00:01
     |████████████████████████████████| 4.7 MB 12.1 MB/s eta 0:00:01
     |████████████████████████████████| 301 kB 12.0 MB/s eta 0:00:01
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.3.2
    Uninstalling matplotlib-3.3.2:
      Successfully uninstalled matplotlib-3.3.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.2
    Uninstalling scipy-1.5.2:
      Successfully uninstalled scipy-1.5.2
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

scipy 1.10.1 requires numpy<1.27.0,>=1.19.5,

In [2]:
from ase.io import read
from ase.calculators.espresso import Espresso
from ase.optimize import BFGS

/home/oem/anaconda3/envs/phy20/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from ase.calculators.espresso import EspressoProfile

In [4]:
import sys, os

In [5]:
sys.path

['/home/phy20',
 '/home/oem/anaconda3/envs/phy20/lib/python38.zip',
 '/home/oem/anaconda3/envs/phy20/lib/python3.8',
 '/home/oem/anaconda3/envs/phy20/lib/python3.8/lib-dynload',
 '',
 '/home/oem/anaconda3/envs/phy20/lib/python3.8/site-packages',
 '/home/oem/anaconda3/envs/phy20/lib/python3.8/site-packages/IPython/extensions',
 '/home/phy20/.ipython']

In [28]:
from ase.io import read

def convert_cif_to_qe_input(cif_file, output_file, pseudopotentials, prefix="PO4"):
    """
    CIF 파일을 Quantum ESPRESSO 입력 파일로 변환
    """
    structure = read(cif_file)  # CIF 파일 읽기

    # Quantum ESPRESSO 입력 파일 작성
    with open(output_file, "w") as f:
        f.write(f"""
&control
    calculation = 'vc-relax',
    outdir = './',
    pseudo_dir = './',
    prefix = '{prefix}',
/
&system
    ibrav = 0,
    nat = {len(structure)},  ! 원자 개수
    ntyp = {len(set(structure.get_chemical_symbols()))},  ! 원소 종류
    ecutwfc = 30.0,
    ecutrho = 240.0,
    occupations = 'smearing',
    smearing = 'gaussian',
    degauss = 0.01,
/
&electrons
    conv_thr = 1.0d-8,
/
&ions
/
&cell
    cell_dofree = 'all',
/

ATOMIC_SPECIES
{pseudopotentials}

CELL_PARAMETERS angstrom
  10.0   0.0   0.0
   0.0  10.0   0.0
   0.0   0.0  10.0

ATOMIC_POSITIONS angstrom
""")
        for atom in structure:
            f.write(f"{atom.symbol} {atom.position[0]:.6f} {atom.position[1]:.6f} {atom.position[2]:.6f}\n")
        f.write("\nK_POINTS automatic\n4 4 4 1 1 1\n")

In [29]:
import subprocess

def run_qe(input_file, output_file):
    """
    Quantum ESPRESSO 실행
    """
    qe_command = f"/home/oem/qe-6.7/bin/pw.x < {input_file} > {output_file}"
    subprocess.run(qe_command, shell=True)

In [30]:
def parse_qe_output(output_file):
    """
    Quantum ESPRESSO 결과 파일 분석
    """
    cell_parameters = []
    total_energy = None

    with open(output_file, "r") as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if "CELL_PARAMETERS" in line:
                cell_parameters = [
                    [float(x) for x in lines[i + 1].split()],
                    [float(x) for x in lines[i + 2].split()],
                    [float(x) for x in lines[i + 3].split()],
                ]
            if "!" in line and "total energy" in line:
                total_energy = float(line.split()[-2])  # 총 에너지 추출
    return cell_parameters, total_energy

In [31]:
# 1. CIF 파일에서 Quantum ESPRESSO 입력 파일 생성
cif_file = "PO4.cif"
qe_input_file = "PO4-vc_relax.in"
qe_output_file = "PO4-vc_relax.out"
pseudopotentials = """
P   30.97376   P.pbe-n-kjpaw_psl.1.0.0.UPF
O   15.9994    O.pbe-n-kjpaw_psl.1.0.0.UPF
"""
convert_cif_to_qe_input(cif_file, qe_input_file, pseudopotentials)

# 2. Quantum ESPRESSO 실행
run_qe(qe_input_file, qe_output_file)

# 3. 결과 파일 분석
cell_params, energy = parse_qe_output(qe_output_file)

print("최종 셀 매개변수:")
for row in cell_params:
    print(row)

print(f"총 에너지: {energy} Ry")

최종 셀 매개변수:
[5.909995089, -0.039872721, 0.0]
[-0.040384821, 6.04074439, 0.0]
[0.0, 0.0, 5.234971861]
총 에너지: -221.77644624 Ry


In [11]:
!less PO4-vc_relax.out

PO4-vc_relax.out (END)...skipping...
~
~
~
~
~
~
~
~
~
~
~
~
~
~
~
~
~
~
~
~
~
~
~
(END)